In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [3]:
from fl_g13.config import RAW_DATA_DIR


from torchvision import datasets, transforms

from fl_g13 import dataset as dataset_handler

import torch
import torch.nn as nn
import torch.nn.functional as F


import flwr
from flwr.common import Context

from flwr.simulation import run_simulation


2025-04-20 19:08:24.280 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13


In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
# disable_progress_bar()

Training on cuda
Flower 1.17.0 / PyTorch 2.6.0+cu118


# Load data

In [5]:
transform = transforms.Compose([
    transforms.ToTensor()
])
cifar100_train = datasets.CIFAR100(root=RAW_DATA_DIR, train=True, download=True, transform=transform)
cifar100_test = datasets.CIFAR100(root=RAW_DATA_DIR, train=False, download=True, transform=transform)

In [6]:
### train val split
train_dataset,val_dataset = dataset_handler.train_test_split(cifar100_train,train_ratio=0.8)

In [7]:
# I.I.D Sharding Split
## k client
k =10
clients_dataset_train= dataset_handler.iid_sharding(train_dataset,k)
clients_dataset_val= dataset_handler.iid_sharding(val_dataset,k)

## Tiny model

In [8]:
class TinyCNN(nn.Module):
    def __init__(self, num_classes=100):
        super(TinyCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))     # -> [B, 16, 32, 32]
        x = F.max_pool2d(x, 2)        # -> [B, 16, 16, 16]
        x = F.relu(self.conv2(x))     # -> [B, 32, 16, 16]
        x = F.max_pool2d(x, 2)        # -> [B, 32, 8, 8]
        x = x.view(x.size(0), -1)     # -> [B, 32*8*8]
        x = self.fc1(x)               # -> [B, 100]
        return x

## Init model , optimizer and loss function

In [9]:
from torch.nn import CrossEntropyLoss
from fl_g13.architectures import BaseDino
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.optim import SGD

# net = TinyCNN().to(DEVICE)
# # optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
# optimizer = torch.optim.AdamW(net.parameters(), lr=1e-4, weight_decay=0.04)
# criterion = torch.nn.CrossEntropyLoss()

start_epoch=1
num_epochs=50
save_every=1
backup_every=10

# Hyper-parameters
BATCH_SIZE = 128
LR = 1e-2


# Model
model = BaseDino()
model.to(DEVICE)
print(f"Model: {model}")

# Optimizer, scheduler, and loss function
optimizer = SGD(model.parameters(), lr=LR)
scheduler = CosineAnnealingWarmRestarts(
    optimizer, 
    T_0=8,           # First restart after 8 epochs
    T_mult=2,        # Double the interval between restarts each time
    eta_min=1e-5     # Minimum learning rate after annealing
)
criterion = CrossEntropyLoss()

Using cache found in C:\Users\ADMIN/.cache\torch\hub\facebookresearch_dino_main


Model: BaseDino(
  (net): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.1, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.1, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (norm): Layer

# Define the ClientApp

## Build module local

Build module local such that ClientApp can use it

In [14]:
!pip install -e ..

Obtaining file:///C:/Users/ADMIN/Desktop/BACKUP/study/Italy/polito/classes/20242/deep%20learning/project/source_code/fl-g13
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for fl_g13 (pyproject.toml): started
  Building editable for fl_g13 (pyproject.toml): finished with status 'done'
  Created wheel for fl_g13: filename=fl_g13-0.0.1-py3-none-any.whl size=4649 sha256=a588eeccbccdbea56dbfe92e55bd612babd81e07366f39e361ccc99a5ffdec08
  Stored in directory: C:\Users\ADMIN\AppData\Local\Temp\pip-ephem-wheel-cache-we99gvxy\wheels\b7

## create FlowerClient instances  

In [11]:
'''
Function load data client is to simulate the distribution data into each client
In the real case, each client will have its dataset
'''
def load_data_client(context: Context):
    partition_id = context.node_config["partition-id"] 
    print(f"Client {partition_id} is ready to train")
    trainloader = DataLoader(clients_dataset_train[partition_id])
    valloader = DataLoader(clients_dataset_val[partition_id])
    return trainloader, valloader

### Create instant of ClientApp

In [12]:
from fl_g13.fl_pytorch.client_app import get_client_app

config={'local-epochs':1}
client = get_client_app(load_data_client,model=model,optimizer=optimizer,criterion=criterion,device=DEVICE,config=config)

# Define the Flower ServerApp

Customize built-in strategy Federated Averaging (FedAvg) of Flower to combine hyperparams in server-side and save model for each k epoch

The strategy could also incremental training an

## Create instant of ServerApp

In [15]:

from pathlib import Path
from torch.utils.data import DataLoader
from fl_g13.fl_pytorch.server_app import get_server_app

def get_datatest_fn(context: Context):
    return DataLoader(cifar100_test)

## checkpoints directory
current_path = Path.cwd()
model_test_path = current_path / "../models/fl_baseline"
model_test_path.resolve()


num_rounds=2
save_every =1
fraction_fit=1.0  # Sample 100% of available clients for training
fraction_evaluate=0.5  # Sample 50% of available clients for evaluation
min_fit_clients=10  # Never sample less than 10 clients for training
min_evaluate_clients=5  # Never sample less than 5 clients for evaluation
min_available_clients=10  # Wait until all 10 clients are available
device=DEVICE
use_wandb=False


server = get_server_app(checkpoint_dir=model_test_path.resolve(),
                        model_class=BaseDino,
                        optimizer=optimizer,
                        criterion=criterion, 
                        scheduler=scheduler,
                        get_datatest_fn=get_datatest_fn,
                        num_rounds=num_rounds,
                        fraction_fit=fraction_fit, 
                        fraction_evaluate=fraction_evaluate,  
                        min_fit_clients=min_fit_clients,  
                        min_evaluate_clients=min_evaluate_clients, 
                        min_available_clients=min_available_clients, 
                        device=device,
                        use_wandb=use_wandb,
                        save_every=save_every
                        )

🔍 Loading checkpoint from C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\fl_baseline\FL_BaseDino_epoch_1.pth
📦 Model class in checkpoint: BaseDino
🔧 Model configuration: {'variant': 'dino_vits16', 'dropout_rate': 0.1, 'head_hidden_size': 1024, 'head_layers': 5, 'num_classes': 100, 'unfreeze_blocks': 3, 'activation_fn': 'GELU', 'pretrained': True}


Using cache found in C:\Users\ADMIN/.cache\torch\hub\facebookresearch_dino_main
Using cache found in C:\Users\ADMIN/.cache\torch\hub\facebookresearch_dino_main


✅ Loaded checkpoint from C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\fl_baseline\FL_BaseDino_epoch_1.pth, resuming at epoch 2


# Run the training


In [16]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if DEVICE == "cuda":
    backend_config["client_resources"]= {"num_cpus": 1, "num_gpus": 1}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

### Download missing module for clients

Dino model,that is serialized and sent to client by server, require some modules that have to download from source code of dino model


In [21]:
import os
import urllib.request

def download_if_not_exists(file_path: str, file_url: str):
    """
    Checks if a file exists at the given path. If it does not, downloads it from the specified URL.

    Parameters:
    - file_path (str): The local path to check and save the file.
    - file_url (str): The URL from which to download the file.
    """
    if not os.path.exists(file_path):
        print(f"'{file_path}' not found. Downloading from {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
            print("Download complete.")
        except Exception as e:
            print(f"Failed to download file: {e}")
    else:
        print(f"'{file_path}' already exists.")

In [22]:
download_if_not_exists("vision_transformer.py", "wget https://raw.githubusercontent.com/facebookresearch/dino/refs/heads/main/vision_transformer.py")
download_if_not_exists("utils.py", "wget https://raw.githubusercontent.com/facebookresearch/dino/refs/heads/main/utils.py")


'vision_transformer.py' already exists.
'utils.py' already exists.


In [23]:
NUM_CLIENTS =10

In [24]:
# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)

Continue train model from epoch 2


INFO :      Starting Flower ServerApp, config: num_rounds=2, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      ROUND 0💡 New best global model found: 0.010000
INFO :      initial parameters (loss, other metrics): 5.092082308197021, {'centralized_accuracy': 0.01}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)
(ClientAppActor pid=10356) 2025-04-20 21:23:43.310 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13


(ClientAppActor pid=10356) Client 0 is ready to train
(ClientAppActor pid=10356) No prefix/name for the model was provided, choosen prefix/name: itchy_kakuna_98
(ClientAppActor pid=10356) 
(ClientAppActor pid=10356) 🚀 Epoch 1/1 (100.00%) Completed
(ClientAppActor pid=10356) 	📊 Training Loss: 5.3544
(ClientAppActor pid=10356) 	✅ Training Accuracy: 0.90%
(ClientAppActor pid=10356) 	⏳ Elapsed Time: 44.55s | ETA: 0.00s
(ClientAppActor pid=10356) 	🕒 Completed At: 21:24
(ClientAppActor pid=10356) 
(ClientAppActor pid=10356) Client 1 is ready to train
(ClientAppActor pid=10356) No prefix/name for the model was provided, choosen prefix/name: jazzy_pikachu_20
(ClientAppActor pid=10356) 
(ClientAppActor pid=10356) 🚀 Epoch 1/1 (100.00%) Completed
(ClientAppActor pid=10356) 	📊 Training Loss: 5.3812
(ClientAppActor pid=10356) 	✅ Training Accuracy: 0.92%
(ClientAppActor pid=10356) 	⏳ Elapsed Time: 42.50s | ETA: 0.00s
(ClientAppActor pid=10356) 	🕒 Completed At: 21:25
(ClientAppActor pid=10356) 
(Clie

INFO :      aggregate_fit: received 10 results and 0 failures


Saving centralized model epoch 2 aggregated_parameters...
💾 Saved checkpoint at: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\fl_baseline\FL_BaseDino_epoch_2.pth


...

INFO :      aggregate_fit: received 10 results and 0 failures


Saving centralized model epoch 3 aggregated_parameters...
💾 Saved checkpoint at: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\fl_baseline\FL_BaseDino_epoch_3.pth


INFO :      fit progress: (2, 4.645598405885696, {'centralized_accuracy': 0.01}, 1233.8158471999996)
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)


(ClientAppActor pid=10356) Client 0 is ready to train
(ClientAppActor pid=10356) Client 3 is ready to train
(ClientAppActor pid=10356) Client 4 is ready to train
(ClientAppActor pid=10356) Client 7 is ready to train
(ClientAppActor pid=10356) Client 9 is ready to train


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 2 round(s) in 1273.77s
INFO :      	History (loss, distributed):
INFO :      		round 1: 4.867536987304687
INFO :      		round 2: 4.804164573812485
INFO :      	History (loss, centralized):
INFO :      		round 0: 5.092082308197021
INFO :      		round 1: 4.665144852542877
INFO :      		round 2: 4.645598405885696
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'federated_evaluate_accuracy': [(1, 0.0102), (2, 0.0114)]}
INFO :      	History (metrics, centralized):
INFO :      	{'centralized_accuracy': [(0, 0.01), (1, 0.0086), (2, 0.01)]}
INFO :      
